## Aeysha SURP CITA Course Project

The goal of the SURP project is to determine if intermediate mass black holes (IMBHs) in Milky Way globular clusters (GCs) are a significant source of hypervelocity stars (HVSs) in the Milky Way. How many fast stars should there be out there? How do they depend on the properties of the cluster or IMBH that ejected them? Are they detectable today or in the near future with the observational facilities we have? [Here](https://lweb.cfa.harvard.edu/~wbrown/Files/ARAA2015.pdf) is a useful review on HVSs. Sections 5-7 are fairly outdated but 1-4 have aged well.

We'll start with a different situation already implemented in `speedystar`, where stars are being ejected following interactions with Sgr A* in the Galactic Centre. Changing assumptions about the ejection model and seeing what impact that has on the HVS population will give you some practice poking around and making changes to the code, analyzing its outputs and get you thinking about how the situation will differ when we consider ejections from GCs.

In [ ]:
# Install needed packages
pip install astropy tqdm scipy galpy mwdust pygaia

In [ ]:
# Import needed packages, libraries and functions
import numpy as np
from speedystar import starsample
from speedystar.eject import Hills
from speedystar.utils.mwpotential import MWPotential
from galpy.potential.mwpotentials import MWPotential2014
from galpy import potential
import mwdust
from astropy.table import Table
from astropy import units as u
import matplotlib.pyplot as plt

### 1 Characterizing a sample of Hills-ejected stars

* Eject, propagate and do the photometry for a sample of Hills mechanism-ejected stars as in the example notebook with all the default arguments (Do not do the mysample.GCv > mysample.vesc cut as in the example). Examine the .fits file and make a plot of the Galactocentric velocity (GCv) vs distance (GCdist). How far away and how fast are the ejected stars?
    * Hint: The default ejection rate for the Hills mechanism here is 1e-4/u.yr. If you want more points to make a prettier plot, you can change the `rate` argument in `speedystar.eject.Hills()` to a larger value (remember it still needs the astropy units)

* mysample.vesc is the escape velocity for each star at its position. Plot escape velocity vs. Galactocentric distance for these stars. How does the escape velocity change with increasing distance?

* The ejected stars are only hypervelocity stars if they are faster than the escape velocity. Make a mysample.GCv > mysample.Vesc cut. How many hypervelocity stars are in this sample?

* Most of these stars will be so faint and far away that they are practically impossible to detect. Make a 'Gaia_DR3' subsample cut as in the example notebook to find only the stars that will be detectable in Gaia DR3. How many hypervelocity stars are in this sample? How do their distances and velocities differ from the overall HVS sample?

* BONUS: What is the definition of escape velocity in this context? How would you go about calculating it if you know the form of the Galactic potential? 
    * Hint: `speedystar.dynamics.get_vesc` is the function that calculates the escape velocity for each star, you can track down its documentation for a hint.

In [ ]:
# EJECTING THE SAMPLE

ejectionmodel = Hills()

# Eject a sample of stars from Sgr A*
sample = starsample(ejectionmodel, name='Catalogue of Sample Stars')

# Save ejection sample
sample.save('./sample_ejection.fits')

In [ ]:
# PROPAGATING THE SAMPLE

# Load ejection sample, if it doesn't already exist
sample = starsample('./sample_ejection.fits')

# Assume a Galactic potential
gal_potential = MWPotential2014

# Make sure the potential has physical units so that the final positions and velocities have physical units too
potential.turn_physical_on(gal_potential)

# Propagate sample
sample.propagate(potential = gal_potential)

# Save propagated sample
sample.save('./sample_propagated.fits')

In [ ]:
# DO THE PHOTOMETRY FOR THE SAMPLE

# Load a pre-existing propagated sample, if needed
sample = starsample('./sample_propagated.fits')

# Assign the dust map
sample.dust = mwdust.Combined15()

# Get mock apparent magnitudes
# By default, magnitudes are computed in the Johnson-Cousins V and I bands and the Gaia G, G_RP, G_BP and G_RVS bands
sample.photometry()

# Save the sample with mock photometry
sample.save('./sample_photometry.fits')

In [ ]:
# EXAMINE THE .FITS FILE AND MAKE A PLOT OF GCv VS GCdist

# Get galactocentric distance and velocity data from .fits file
tab = Table.read('sample_photometry.fits')

GCdist = tab['GCdist'] 
GCv = tab['GCv'] 

# Plot galactocentric velocity vs distance 
plt.scatter(GCdist, GCv, color='tab:blue', s=10)
plt.xlabel('galactocentric distance (kpc)') 
plt.ylabel('galactocentric velocity (km/s)') 
plt.title('galactocentric velocity vs distance for a sample of stars ejected by Sgr A*')
plt.grid(True)

# Save plot as PDF
plt.savefig("GCv_vs_GCdist_1.pdf", format="pdf", bbox_inches="tight")

plt.show()

In [ ]:
# PLOT V_ESC VS GCdist

# Get the escape velocity for each star at its position
v_esc = sample.Vesc 

# Plot escape velocity vs galactocentric distance 
plt.scatter(GCdist, v_esc, color='tab:blue', s=10)
plt.xlabel('galactocentric distance (kpc)') 
plt.ylabel('escape velocity (km/s)') 
plt.title('escape velocity vs distance for a sample of stars ejected by Sgr A*')
plt.grid(True)

# Save plot as PDF
plt.savefig("Vesc_vs_GCdist.pdf", format="pdf", bbox_inches="tight")

plt.show()

In [ ]:
# FIND HYPERVELOCITY STARS

print("Total number of stars:", sample.size)

# Use boolean indexing to make a cut of the sample, keeping only hypervelocity stars 
hyp_vel = sample.GCv > sample.Vesc
sample = sample[hyp_vel]

print("Number of hypervelocity stars:", sample.size)

GCdist = GCdist[hyp_vel]
GCv = GCv[hyp_vel]

# Plot galactocentric velocity vs distance for only the hypervelocity stars in the sample
plt.scatter(GCdist, GCv, color='tab:blue', s=10)
plt.xlabel('galactocentric distance (kpc)') 
plt.ylabel('galactocentric velocity (km/s)') 
plt.title('galactocentric velocity vs distance for a sample of hypervelocity stars ejected by Sgr A*')
plt.grid(True)

# Save plot as PDF
plt.savefig("GCv_vs_GCdist_hyp_1a.pdf", format="pdf", bbox_inches="tight")

plt.show()

In [ ]:
# FIND ONLY THE STARS DETECTABLE IN GAIA DR3

# Load a pre-existing sample with photometry, if needed
sample = starsample('./sample_photometry.fits')

# Determine which stars would be in principle detectable in Gaia DR3 and recalculate errors
sample.subsample('Gaia_DR3')

# Save the cut sample
sample.save('./sample_gaiaDR3.fits')

print("Number of stars in Gaia DR3:", sample.size)

In [ ]:
# FIND THE HYPERVELOCITY STARS IN THE GAIA DR3 SUBSAMPLE

# Use boolean indexing to make a cut of the sample, keeping only hypervelocity stars 
hyp_vel = sample.GCv > sample.Vesc
sample = sample[hyp_vel]

print("Number of hypervelocity stars in Gaia DR3:", sample.size) 

# Get galactocentric distance and velocity data from .fits file
tab = Table.read('sample_gaiaDR3.fits')
GCdist = tab['GCdist']
GCv = tab['GCv']

# Plot galactocentric velocity vs distance data for only the hypervelocity stars in the Gaia DR3 subsample
plt.scatter(GCdist, GCv, color='tab:blue', s=10)
plt.xlabel('galactocentric distance (kpc)') 
plt.ylabel('galactocentric velocity (km/s)') 
plt.title('galactocentric velocity vs distance for a sample of hypervelocity, Gaia DR3 detectable stars ejected by Sgr A*')
plt.grid(True)

# Save plot as PDF
plt.savefig("GCv_vs_GCdist_hyp_Gaia_DR3.pdf", format="pdf", bbox_inches="tight")

plt.show()

In [ ]:
# COMPARE THE TOTAL HVS SAMPLE TO THE GAIA DR3 HVS SAMPLE

# Load photometry sample, if it doesn't already exist
sample = starsample('./sample_photometry.fits')

# Get galactocentric distance and velocity data for only the hypervelocity stars
tab1 = Table.read('sample_photometry.fits')

hyp_vel = sample.GCv > sample.Vesc

GCdist1 = tab1['GCdist']
GCdist1 = GCdist1[hyp_vel]

GCv1 = tab1['GCv']
GCv1 = GCv1[hyp_vel]

# Load photometry sample for the Gaia DR3 subsample, if it doesn't already exist
sample = starsample('./sample_gaiaDR3.fits')

# Get galactocentric distance and velocity data for only the hypervelocity stars in the Gaia DR3 subsample
tab2 = Table.read('sample_gaiaDR3.fits')

hyp_vel = sample.GCv > sample.Vesc

GCdist2 = tab2['GCdist']
GCdist2 = GCdist2[hyp_vel]

GCv2 = tab2['GCv']
GCv2 = GCv2[hyp_vel]

# Plot galactocentric velocity vs distance for the hypervelocity stars
plt.scatter(GCdist1, GCv1, color='tab:blue', s=10, label='total HVS sample')
plt.scatter(GCdist2, GCv2, color='tab:red', s=10, label='Gaia DR3 HVS sample')
plt.xlabel('galactocentric distance (kpc)') 
plt.ylabel('galactocentric velocity (km/s)') 
plt.title('galactocentric velocity vs distance for a sample of hypervelocity stars ejected by Sgr A*')
plt.grid(True)
plt.legend(loc='lower right')

# Save plot as PDF
plt.savefig("GCv_vs_GCdist_hyp_1b.pdf", format="pdf", bbox_inches="tight")

plt.show()

### 2 The impact of the black hole mass

* One way ejecting stars from IMBHs in GCs is going to be different from ejection stars from Sgr A* is that IMBHs are much less massive than Sgr A*. Figure out how to change the black hole mass in `speedystar.eject.Hills()` to 4e4 M_sun. Generate another catalogue and see how many hypervelocity stars you get. How do the samples differ?
    * BONUS: The ejection velocity of a star depends analytically on the black hole mass. In the [review article mentioned above](https://lweb.cfa.harvard.edu/~wbrown/Files/ARAA2015.pdf) or directly in `speedystar.eject.Hills()`, find what that dependence is. If you reduce the black hole mass by a factor of 100, how should the ejection velocity change?

In [ ]:
# EJECTING THE SAMPLE FROM AN IMBH

ejectionmodel = Hills(M_BH=4e4*u.Msun) # Change black hole mass to 4e4 M_sun

# Eject a sample of stars from IMBH 
sample_IMBH = starsample(ejectionmodel, name='Catalogue of Sample Stars (ejected from IMBH)')

# Save ejection sample
sample_IMBH.save('./sample_IMBH_ejection.fits')

In [ ]:
# PROPAGATING THE IMBH SAMPLE 

# Load ejection sample, if it doesn't already exist
sample_IMBH = starsample('./sample_IMBH_ejection.fits')

# Assume a Galactic potential
gal_potential = MWPotential2014

# Make sure the potential has physical units so that the final positions and velocities have physical units too
potential.turn_physical_on(gal_potential)

# Propagate sample
sample_IMBH.propagate(potential = gal_potential)

# Save propagated sample
sample_IMBH.save('./sample_IMBH_propagated.fits')

In [ ]:
# DO THE PHOTOMETRY FOR THE IMBH SAMPLE 

# Load a pre-existing propagated sample, if needed
sample_IMBH = starsample('./sample_IMBH_propagated.fits')

# Assign the dust map
sample_IMBH.dust = mwdust.Combined15()

# Get mock apparent magnitudes. By default magnitudes are computed in the Johnson-Cousins V and I bands 
# and the Gaia G, G_RP, G_BP and G_RVS bands
sample_IMBH.photometry()

# Save the sample with mock photometry
sample_IMBH.save('./sample_IMBH_photometry.fits')

In [ ]:
# FIND HYPERVELOCITY STARS IN THE IMBH SAMPLE

print("Total number of stars:", sample_IMBH.size)

# Use boolean indexing to make a cut of the sample, keeping only hypervelocity stars 
hyp_vel = sample_IMBH.GCv > sample_IMBH.Vesc
sample_IMBH = sample_IMBH[hyp_vel]

print("Number of hypervelocity stars:", sample_IMBH.size)

# Get galactocentric distance and velocity data from .fits file
tab = Table.read('sample_IMBH_photometry.fits')
GCdist = tab['GCdist']
GCv = tab['GCv']

GCdist = GCdist[hyp_vel]
GCv = GCv[hyp_vel]

# Plot galactocentric velocity vs distance for only the hypervelocity stars in the sample
plt.scatter(GCdist, GCv, color='tab:blue', s=10)
plt.xlabel('galactocentric distance (kpc)') 
plt.ylabel('galactocentric velocity (km/s)') 
plt.title('galactocentric velocity vs distance for a sample of hypervelocity stars ejected by IMBH')
plt.grid(True)

# Save plot as PDF
plt.savefig("GCv_vs_GCdist_hyp_2.pdf", format="pdf", bbox_inches="tight")

plt.show()

### 3 The impact of the initial mass function

* Another way the Galactic Centre differs from GCs is that its stellar initial mass function (IMF) is known to be top-heavy, i.e. the proportion of massive stars is larger than you'd expect to find in more typical areas of the Galaxy.
* The slope of the IMF in the ejection model is set by an argument called `kappa` in `speedystar.eject.Hills()`. By default it is set to -2.35. A value of -1.7 is [probably more accurate](https://arxiv.org/abs/1301.0540). Set kappa to -1.7 and eject another sample. How does the HVS population change?
    * Bonus: another way the Galactic Centre and GCs differ is that the Galactic Centre is still star-forming while GCs stopped forming stars billions of years ago. Keep that in your head, because it'll be important later.

In [ ]:
# EJECTING THE GC SAMPLE 

ejectionmodel = Hills(kappa=-1.7) # Change the slope of the IMF to -1.7

# Eject a sample of stars from the GC
sample_GC = starsample(ejectionmodel, name='Catalogue of Sample Stars (ejected from GC)')

# Save ejection sample
sample_GC.save('./sample_GC_ejection.fits')

In [ ]:
# PROPAGATING THE GC SAMPLE

# Load ejection sample, if it doesn't already exist
sample_GC = starsample('./sample_GC_ejection.fits')

# Assume a Galactic potential
gal_potential = MWPotential2014

# Make sure the potential has physical units so that the final positions and velocities have physical units too
potential.turn_physical_on(gal_potential)

# Propagate sample 
sample_GC.propagate(potential = gal_potential)

# Save propagated sample
sample_GC.save('./sample_GC_propagated.fits')

In [ ]:
# DO THE PHOTOMETRY FOR THE GC SAMPLE 

# Load a pre-existing propagated sample, if needed
sample_GC = starsample('./sample_GC_propagated.fits')

# Assign the dust map
sample_GC.dust = mwdust.Combined15()

# Get mock apparent magnitudes. By default magnitudes are computed in the Johnson-Cousins V and I bands 
# and the Gaia G, G_RP, G_BP and G_RVS bands.
sample_GC.photometry()

# Save the sample with mock photometry
sample_GC.save('./sample_GC_photometry.fits')

In [ ]:
# FIND HYPERVELOCITY STARS IN THE GC SAMPLE 

print("Total number of stars:", sample_GC.size)

# Use boolean indexing to make a cut of the sample, keeping only hypervelocity stars 
hyp_vel = sample_GC.GCv > sample_GC.Vesc
sample_GC = sample_GC[hyp_vel]

print("Number of hypervelocity stars:", sample_GC.size)

# Get effective temperature and mass data from .fits file
tab = Table.read('sample_GC_photometry.fits')
T_eff = tab['T_eff']
m = tab['m'] 

T_eff = T_eff[hyp_vel]
m = m[hyp_vel]

# Plot effective temperature vs mass for only the hypervelocity stars in the sample
plt.scatter(m, T_eff, color='tab:blue', s=10)
plt.xlabel('Mass ($M_{\odot}$)') 
plt.ylabel('$T_{eff}$ (K)') 
plt.title('$T_{eff}$ vs mass for a sample of hypervelocity stars ejected from GC')
plt.grid(True)

# Save plot as PDF
plt.savefig("Teff_vs_mass_hyp_3.pdf", format="pdf", bbox_inches="tight")

plt.show()

### 4 The impact of stellar metallicity

* Yet another way the Galactic Centre differs from GCs is in the metallicity of the ejected stars. The default in `speedystar.eject.Hills()` is to assume all stars have the same metallicity as the Sun. In reality, stars in the Galactic Centre have a range of metallicites and are probably on average [slightly more metal-rich](https://ui.adsabs.harvard.edu/abs/2020MNRAS.494..396F/abstract) than the Sun. Stars in GCs, on the other hand, have metallicities an order of magnitude (or lower) than the Sun. This impacts the physical sizes and lifetimes of the stars, so their maximum flight times and maximum ejection velocities are lower and they will have different apparent magnitudes.

* Figure out how to change the assumed metallicity in `speedystar.eject.Hills()` and set it to -1, i.e. metallicity 1/10th of Solar. How does the HVS population change?

In [ ]:
# EJECTING THE LOW METALLICITY SAMPLE

ejectionmodel = Hills(Met=-1.0) # Change the assumed metallicity to -1.0

# Eject a sample of stars
sample_low_metal = starsample(ejectionmodel, name='Catalogue of Sample Stars (low metallicity)')

# Save ejection sample
sample_low_metal.save('./sample_low_metal_ejection.fits')

In [ ]:
# PROPAGATING THE LOW METALLICITY SAMPLE

# Load ejection sample, if it doesn't already exist
sample_low_metal = starsample('./sample_low_metal_ejection.fits')

# Assume a Galactic potential
gal_potential = MWPotential2014

# Make sure the potential has physical units so that the final positions and velocities have physical units too
potential.turn_physical_on(gal_potential)

# Propagate sample
sample_low_metal.propagate(potential = gal_potential)

# Save propagated sample
sample_low_metal.save('./sample_low_metal_propagated.fits')

In [ ]:
# DO THE PHOTOMETRY FOR THE LOW METALLICITY SAMPLE

# Load a pre-existing propagated sample, if needed
sample_low_metal = starsample('./sample_low_metal_propagated.fits')

# Assign the dust map
sample_low_metal.dust = mwdust.Combined15()

# Get mock apparent magnitudes
sample_low_metal.photometry()

# Save the sample with mock photometry
sample_low_metal.save('./sample_low_metal_photometry.fits')

In [ ]:
# FIND HYPERVELOCITY STARS IN THE LOW METALLICITY SAMPLE

print("Total number of stars:", sample_low_metal.size)

# Use boolean indexing to make a cut of the sample, keeping only hypervelocity stars 
hyp_vel = sample_low_metal.GCv > sample_low_metal.Vesc
sample_low_metal = sample_low_metal[hyp_vel]

print("Number of hypervelocity stars:", sample_low_metal.size)

# Get effective temperature and mass data from .fits file
tab = Table.read('sample_low_metal_photometry.fits')
T_eff = tab['T_eff']
m = tab['m'] # mass in solar mass units

T_eff = T_eff[hyp_vel]
m = m[hyp_vel]

# Plot effective temperature vs mass for only the hypervelocity stars in the sample
plt.scatter(m, T_eff, color='tab:blue', s=10)
plt.xlabel('Mass ($M_{\odot}$)') 
plt.ylabel('$T_{eff}$ (K)') 
plt.title('$T_{eff}$ vs mass for a sample of low metallicity, hypervelocity stars ejected from GC')
plt.grid(True)

# Save plot as PDF
plt.savefig("Teff_vs_mass_hyp_4.pdf", format="pdf", bbox_inches="tight")

plt.show()

### 5 BONUS: The impact of the Galactic potential

* Something we've ignored so far is the gravitational potential of the Galaxy itself. There still exists considerable uncertainty about the strength and form of the potential, particularly at large distances, and this affects the HVS population.

* We've so far used MWPotential2014, which is a commonly used potential for the Milky Way. There is a function `speedystar.util.mwpotential.MWPotential()` that implements a different reasonable potential as described [here](https://arxiv.org/abs/1711.11397). Import this function and eject two samples, one propagated through this potential with its default arguments and one in which the Milky Way halo mass is doubled. How does the HVS population change?

In [ ]:
# EJECTING THE SAMPLE WITH A DIFFERENT POTENTIAL

ejectionmodel = Hills() 

# Eject a sample of stars 
sample_diff_pot_1 = starsample(ejectionmodel, name='Catalogue of Sample Stars (different potential)')

# Save ejection sample
sample_diff_pot_1.save('./sample_diff_pot_1_ejection.fits')

In [ ]:
# PROPAGATING THE SAMPLE WITH A DIFFERENT POTENTIAL

# Load ejection sample, if it doesn't already exist
sample_diff_pot_1 = starsample('./sample_diff_pot_1_ejection.fits')

# Assume a Galactic potential
gal_potential = MWPotential()

# Make sure the potential has physical units so that the final positions and velocities have physical units too
potential.turn_physical_on(gal_potential)

# Propagate sample
sample_diff_pot_1.propagate(potential = gal_potential)

# Save propagated sample
sample_diff_pot_1.save('./sample_diff_pot_1_propagated.fits')

In [ ]:
# DO THE PHOTOMETRY FOR THE SAMPLE WITH A DIFFERENT POTENTIAL

# Load a pre-existing propagated sample, if needed
sample_diff_pot_1 = starsample('./sample_diff_pot_1_propagated.fits')

# Assign the dust map
sample_diff_pot_1.dust = mwdust.Combined15()

# Get mock apparent magnitudes. By default magnitudes are computed in the Johnson-Cousins V and I bands 
# and the Gaia G, G_RP, G_BP and G_RVS bands.
sample_diff_pot_1.photometry()

# Save the sample with mock photometry
sample_diff_pot_1.save('./sample_diff_pot_1_photometry.fits')

In [ ]:
# FIND HYPERVELOCITY STARS IN THE SAMPLE WITH A DIFFERENT POTENTIAL
print("Total number of stars:", sample_diff_pot_1.size)

# Use boolean indexing to make a cut of the sample, keeping only hypervelocity stars 
hyp_vel = sample_diff_pot_1.GCv > sample_diff_pot_1.Vesc
sample_diff_pot_1 = sample_diff_pot_1[hyp_vel]

print("Number of hypervelocity stars:", sample_diff_pot_1.size)

# Get galactocentric distance and velocity data from .fits file
tab = Table.read('sample_diff_pot_1_photometry.fits')
GCdist = tab['GCdist']
GCv = tab['GCv']

GCdist = GCdist[hyp_vel]
GCv = GCv[hyp_vel]

# Plot galactocentric velocity vs distance for only the hypervelocity stars in the sample
plt.scatter(GCdist, GCv, color='tab:blue', s=10)
plt.xlabel('galactocentric distance (kpc)') 
plt.ylabel('galactocentric velocity (km/s)') 
plt.title('galactocentric velocity vs distance for a sample of hypervelocity stars with a different galactic potential')
plt.grid(True)

# Save plot as PDF
plt.savefig("GCv_vs_GCdist_hyp_5a.pdf", format="pdf", bbox_inches="tight")

plt.show()

In [ ]:
# EJECTING THE SAMPLE WITH A DIFFERENT POTENTIAL AND DOUBLE THE HALO MASS

ejectionmodel = Hills() 

# Eject a sample of stars 
sample_diff_pot_2 = starsample(ejectionmodel, name='Catalogue of Sample Stars (different potential and doubled halo mass)')

# Save ejection sample
sample_diff_pot_2.save('./sample_diff_pot_2_ejection.fits')

In [ ]:
# PROPAGATING THE SAMPLE WITH A DIFFERENT POTENTIAL AND DOUBLE THE HALO MASS

# Load ejection sample, if it doesn't already exist
sample_diff_pot_2 = starsample('./sample_diff_pot_2_ejection.fits')

# Assume a Galactic potential
gal_potential = MWPotential(Ms=2*0.76)

# Make sure the potential has physical units so that the final positions and velocities have physical units too
potential.turn_physical_on(gal_potential)

# Propagate sample
sample_diff_pot_2.propagate(potential = gal_potential)

# Save propagated sample
sample_diff_pot_2.save('./sample_diff_pot_2_propagated.fits')

In [ ]:
# DO THE PHOTOMETRY FOR THE SAMPLE WITH A DIFFERENT POTENTIAL AND DOUBLE THE HALO MASS

# Load a pre-existing propagated sample, if needed
sample_diff_pot_2 = starsample('./sample_diff_pot_2_propagated.fits')

# Assign the dust map
sample_diff_pot_2.dust = mwdust.Combined15()

# Get mock apparent magnitudes. By default magnitudes are computed in the Johnson-Cousins V and I bands 
# and the Gaia G, G_RP, G_BP and G_RVS bands.
sample_diff_pot_2.photometry()

# Save the sample with mock photometry
sample_diff_pot_2.save('./sample_diff_pot_2_photometry.fits')

In [ ]:
# FIND HYPERVELOCITY STARS IN THE SAMPLE WITH A DIFFERENT POTENTIAL AND DOUBLE THE HALO MASS

print("Total number of stars:", sample_diff_pot_2.size)

# Use boolean indexing to make a cut of the sample, keeping only hypervelocity stars 
hyp_vel = sample_diff_pot_2.GCv > sample_diff_pot_2.Vesc
sample_diff_pot_2 = sample_diff_pot_2[hyp_vel]

print("Number of hypervelocity stars:", sample_diff_pot_2.size)

# Get galactocentric distance and velocity data from .fits file
tab = Table.read('sample_diff_pot_2_photometry.fits')
GCdist = tab['GCdist']
GCv = tab['GCv']

GCdist = GCdist[hyp_vel]
GCv = GCv[hyp_vel]

# Plot galactocentric velocity vs distance for only the hypervelocity stars in the sample
plt.scatter(GCdist, GCv, color='tab:blue', s=10)
plt.xlabel('galactocentric distance (kpc)') 
plt.ylabel('galactocentric velocity (km/s)') 
plt.title('galactocentric velocity vs distance for a sample of hypervelocity stars with a different galactic potential and doubled halo mass')
plt.grid(True)

# Save plot as PDF
plt.savefig("GCv_vs_GCdist_hyp_5b.pdf", format="pdf", bbox_inches="tight")

plt.show()